In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Data cleaning and estimating missing values inspired by the following notebooks
* https://www.kaggle.com/code/kenjee/titanic-project-example
* https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial#3.-Model
* https://www.kaggle.com/code/startupsci/titanic-data-science-solutions
* https://www.kaggle.com/code/mohammadabdelhalim/titanic-classification/notebook#The-Challenge


In [2]:
# to ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [3]:
# read in training data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# read in test data
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
train_data.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
test_data.describe()


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [9]:
# number of unique values
train_data.nunique()


PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [10]:
# number of unique values
test_data.nunique()


PassengerId    418
Pclass           3
Name           418
Sex              2
Age             79
SibSp            7
Parch            8
Ticket         363
Fare           169
Cabin           76
Embarked         3
dtype: int64

In [11]:
# check for null values
train_data.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
# check for null values
test_data.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [13]:
# look at survival rates against Passenger class
train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [14]:
# look at survival rates against Sex
train_data[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,Sex,Survived
0,female,0.742038
1,male,0.188908


In [15]:
# look at survival rates against Sibling/Spouse
train_data[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [16]:
# look at survival rates against Parent/Child
train_data[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [17]:
# check the 'Embarked' value counts
train_data["Embarked"].value_counts()


S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [18]:
# check the two missing embarked values in the training data
train_data[train_data['Embarked'].isnull()]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [19]:
# note they are the same ticket, replace the two missing embarked values in the train data woth the most common value
train_data["Embarked"].fillna("S", inplace= True)
train_data.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [20]:
# check the missing fare value
test_data[test_data['Fare'].isnull()]


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [21]:
# check if there are any other passengers with the same ticket
test_data[test_data['Ticket'] == '3701']


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [22]:
# calculate a median fare in 3rd class with no relatives (0 SibSp and ParCh)
median_fare_test_data = test_data.groupby(['Pclass', 'SibSp', 'Parch']).Fare.median()[3][0][0]
median_fare_train_data = train_data.groupby(['Pclass', 'SibSp', 'Parch']).Fare.median()[3][0][0]
median_fare = ((median_fare_test_data + median_fare_train_data) / 2)
print(median_fare)


7.8458000000000006


In [23]:
# use the calculated median fare to fill in the missing value
test_data['Fare'].fillna(median_fare, inplace=True)
test_data.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [24]:
# create a family size variable - FamilySize
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
train_data.sample(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.250,NaN,S,2
719,720,0,3,"Johnson, Mr. Malkolm Joackim",male,33.00,0,0,347062,7.775,NaN,S,1
379,380,0,3,"Gustafsson, Mr. Karl Gideon",male,19.00,0,0,347069,7.775,NaN,S,1
66,67,1,2,"Nye, Mrs. (Elizabeth Ramell)",female,29.00,0,0,C.A. 29395,10.500,F33,S,1
755,756,1,2,"Hamalainen, Master. Viljo",male,0.67,1,1,250649,14.500,NaN,S,3


In [25]:
# create a family size variable - FamilySize
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data.sample(5)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
205,1097,1,"Omont, Mr. Alfred Fernand",male,NaN,0,0,F.C. 12998,25.7417,NaN,C,1
347,1239,3,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,0,0,2688,7.2292,NaN,C,1
225,1117,3,"Moubarek, Mrs. George (Omine Amenia"" Alexander)""",female,NaN,0,2,2661,15.2458,NaN,C,3
248,1140,2,"Hold, Mrs. Stephen (Annie Margaret Hill)",female,29.0,1,0,26707,26.0000,NaN,S,2
93,985,3,"Guest, Mr. Robert",male,NaN,0,0,376563,8.0500,NaN,S,1


In [26]:
# create a solo traveler variable - IsSolo
train_data['IsSolo'] = 0
train_data.loc[(train_data['FamilySize']) == 1, 'IsSolo'] = 1
train_data.sample(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo
166,167,1,1,"Chibnall, Mrs. (Edith Martha Bowerman)",female,NaN,0,1,113505,55.0000,E33,S,2,0
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S,1,1
369,370,1,1,"Aubart, Mme. Leontine Pauline",female,24.0,0,0,PC 17477,69.3000,B35,C,1,1
37,38,0,3,"Cann, Mr. Ernest Charles",male,21.0,0,0,A./5. 2152,8.0500,NaN,S,1,1
756,757,0,3,"Carlsson, Mr. August Sigfrid",male,28.0,0,0,350042,7.7958,NaN,S,1,1


In [27]:
# create a solo traveler variable - IsSolo
test_data['IsSolo'] = 0
test_data.loc[(test_data['FamilySize']) == 1, 'IsSolo'] = 1
test_data.sample(5)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo
346,1238,2,"Botsford, Mr. William Hull",male,26.00,0,0,237670,13.000,NaN,S,1,1
354,1246,3,"Dean, Miss. Elizabeth Gladys Millvina""""",female,0.17,1,2,C.A. 2315,20.575,NaN,S,4,0
123,1015,3,"Carver, Mr. Alfred John",male,28.00,0,0,392095,7.250,NaN,S,1,1
320,1212,3,"Andersson, Mr. Johan Samuel",male,26.00,0,0,347075,7.775,NaN,S,1,1
140,1032,3,"Goodwin, Miss. Jessie Allis",female,10.00,5,2,CA 2144,46.900,NaN,S,8,0


In [28]:
# create Title from Name for training data
train_data['Title'] = [name.split(',')[1].split('.')[0].strip() for name in train_data['Name']]
train_data['Title'].value_counts()


Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [29]:
# break down title to top 4 and label the rest other (Mr, Miss, Mrs, Master, Other)
train_data['Title'] = train_data['Title'].replace(['Dr', 'Rev', 'Mlle', 'Major', 'Col', 'the Countess', 'Capt', 'Ms', 'Sir', 'Lady', 'Mme', 'Don', 'Jonkheer'], 'Other')
train_data['Title'].value_counts()


Mr        517
Miss      182
Mrs       125
Master     40
Other      27
Name: Title, dtype: int64

In [30]:
# create Title from Name for test data
test_data['Title'] = [name.split(',')[1].split('.')[0].strip() for name in test_data['Name']]
test_data['Title'].value_counts()


Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Ms          1
Dr          1
Dona        1
Name: Title, dtype: int64

In [31]:
# break down title to top 4 and label the rest other (Mr, Miss, Mrs, Master, Other)
test_data['Title'] = test_data['Title'].replace(['Col', 'Rev', 'Ms', 'Dr', 'Dona'], 'Other')
test_data['Title'].value_counts()


Mr        240
Miss       78
Mrs        72
Master     21
Other       7
Name: Title, dtype: int64

In [32]:
# change Sex from (male, female) to (0, 1)
train_data['Sex'] = train_data.Sex.map({'male':0, 'female': 1})
train_data.sample(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo,Title
729,730,0,3,"Ilmakangas, Miss. Pieta Sofia",1,25.0,1,0,STON/O2. 3101271,7.9250,NaN,S,2,0,Miss
393,394,1,1,"Newell, Miss. Marjorie",1,23.0,1,0,35273,113.2750,D36,C,2,0,Miss
259,260,1,2,"Parrish, Mrs. (Lutie Davis)",1,50.0,0,1,230433,26.0000,NaN,S,2,0,Mrs
601,602,0,3,"Slabenoff, Mr. Petco",0,NaN,0,0,349214,7.8958,NaN,S,1,1,Mr
339,340,0,1,"Blackwell, Mr. Stephen Weart",0,45.0,0,0,113784,35.5000,T,S,1,1,Mr


In [33]:
# convert Sex from (male, female) to (0, 1) as many models require numerical values and can't use strings
test_data['Sex'] = test_data.Sex.map({'male':0, 'female': 1})
test_data.sample(5)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo,Title
113,1005,3,"Buckley, Miss. Katherine",1,18.5,0,0,329944,7.2833,NaN,Q,1,1,Miss
152,1044,3,"Storey, Mr. Thomas",0,60.5,0,0,3701,7.8458,NaN,S,1,1,Mr
92,984,1,"Davidson, Mrs. Thornton (Orian Hays)",1,27.0,1,2,F.C. 12750,52.0000,B71,S,4,0,Mrs
343,1235,1,"Cardeza, Mrs. James Warburton Martinez (Charlo...",1,58.0,0,1,PC 17755,512.3292,B51 B53 B55,C,2,0,Mrs
382,1274,3,"Risien, Mrs. Samuel (Emma)",1,NaN,0,0,364498,14.5000,NaN,S,1,1,Mrs


In [34]:
# find median age of passengers grouped by Sex and Pclass and fill in those values for nulls
test_data['Age'] = test_data.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
test_data.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
FamilySize       0
IsSolo           0
Title            0
dtype: int64

In [35]:
# find median age of passengers grouped by Sex and Pclass and fill in those values for nulls
train_data['Age'] = train_data.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
train_data.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
FamilySize       0
IsSolo           0
Title            0
dtype: int64

In [36]:
# create bins for age groups
bins = [0, 12, 18, 30, 50, 100]
labels = [0, 1, 2, 3, 4]
train_data['AgeGroup'] = pd.cut(train_data['Age'], bins=bins, labels=labels)
train_data.sample(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo,Title,AgeGroup
305,306,1,1,"Allison, Master. Hudson Trevor",0,0.92,1,2,113781,151.55,C22 C26,S,4,0,Master,0
117,118,0,2,"Turpin, Mr. William John Robert",0,29.00,1,0,11668,21.00,NaN,S,2,0,Mr,2
63,64,0,3,"Skoog, Master. Harald",0,4.00,3,2,347088,27.90,NaN,S,6,0,Master,0
432,433,1,2,"Louch, Mrs. Charles Alexander (Alice Adelaide ...",1,42.00,1,0,SC/AH 3085,26.00,NaN,S,2,0,Mrs,3
790,791,0,3,"Keane, Mr. Andrew ""Andy""",0,25.00,0,0,12460,7.75,NaN,Q,1,1,Mr,2


In [37]:
test_data['AgeGroup'] = pd.cut(test_data['Age'], bins=bins, labels=labels)
test_data.sample(5)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo,Title,AgeGroup
344,1236,3,"van Billiard, Master. James William",0,24.0,1,1,A/5. 851,14.5000,NaN,S,3,0,Master,2
171,1063,3,"Zakarian, Mr. Ortin",0,27.0,0,0,2670,7.2250,NaN,C,1,1,Mr,2
186,1078,2,"Phillips, Miss. Alice Frances Louisa",1,21.0,0,1,S.O./P.P. 2,21.0000,NaN,S,2,0,Miss,2
150,1042,1,"Earnshaw, Mrs. Boulton (Olive Potter)",1,23.0,0,1,11767,83.1583,C54,C,2,0,Mrs,2
206,1098,3,"McGowan, Miss. Katherine",1,35.0,0,0,9232,7.7500,NaN,Q,1,1,Miss,3


In [38]:
# create deck feature from cabin, n represents null
train_data['Deck'] = train_data.Cabin.apply(lambda x: str(x)[0])
train_data['Deck'].value_counts()


n    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Deck, dtype: int64

In [39]:
# convert deck to a numerical value
train_data['Deck'] = train_data.Deck.map({'n': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 1})
train_data.sample(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo,Title,AgeGroup,Deck
530,531,1,2,"Quick, Miss. Phyllis May",1,2.0,1,1,26360,26.0000,NaN,S,3,0,Miss,0,0
48,49,0,3,"Samaan, Mr. Youssef",0,25.0,2,0,2662,21.6792,NaN,C,3,0,Mr,2,0
565,566,0,3,"Davies, Mr. Alfred J",0,24.0,2,0,A/4 48871,24.1500,NaN,S,3,0,Mr,2,0
121,122,0,3,"Moore, Mr. Leonard Charles",0,25.0,0,0,A4. 54510,8.0500,NaN,S,1,1,Mr,2,0
356,357,1,1,"Bowerman, Miss. Elsie Edith",1,22.0,0,1,113505,55.0000,E33,S,2,0,Miss,2,5


In [40]:
# create deck feature from cabin
test_data['Deck'] = test_data.Cabin.apply(lambda x: str(x)[0])
test_data['Deck'].value_counts()


n    327
C     35
B     18
D     13
E      9
F      8
A      7
G      1
Name: Deck, dtype: int64

In [41]:
# convert deck to a numerical value
test_data['Deck'] = test_data.Deck.map({'n': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7})
test_data.sample(5)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsSolo,Title,AgeGroup,Deck
171,1063,3,"Zakarian, Mr. Ortin",0,27.0,0,0,2670,7.2250,NaN,C,1,1,Mr,2,0
6,898,3,"Connolly, Miss. Kate",1,30.0,0,0,330972,7.6292,NaN,Q,1,1,Miss,2,0
229,1121,2,"Hocking, Mr. Samuel James Metcalfe",0,36.0,0,0,242963,13.0000,NaN,S,1,1,Mr,3,0
261,1153,3,"Nilsson, Mr. August Ferdinand",0,21.0,0,0,350410,7.8542,NaN,S,1,1,Mr,2,0
107,999,3,"Ryan, Mr. Edward",0,24.0,0,0,383162,7.7500,NaN,Q,1,1,Mr,2,0


In [42]:
from sklearn.ensemble import RandomForestClassifier

y = train_data['Survived']

features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'IsSolo', 'Title', 'AgeGroup', 'Deck']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

print("Your submission was successfully saved!")


Your submission was successfully saved!
